In [106]:
import pandas as pd
import numpy as np

In [107]:
df = pd.read_csv("./2019-Oct-depth.csv")

In [108]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28933153 entries, 0 to 28933152
Data columns (total 12 columns):
 #   Column         Dtype  
---  ------         -----  
 0   Unnamed: 0     int64  
 1   event_time     object 
 2   event_type     object 
 3   product_id     int64  
 4   category_id    int64  
 5   category_code  object 
 6   brand          object 
 7   price          float64
 8   user_id        int64  
 9   user_session   object 
 10  depth1         object 
 11  depth2         object 
dtypes: float64(1), int64(4), object(7)
memory usage: 2.6+ GB


In [109]:
#여러개 카트에 넣은 경우
df[df["event_type"] == "cart"][["event_type", "product_id", "price", "user_id", "user_session"]].duplicated().value_counts()

False    554998
True     265790
dtype: int64

In [110]:
#여러개 구매한 경우
df[df["event_type"] == "purchase"][["event_type", "product_id", "price", "user_id", "user_session"]].duplicated().value_counts()

False    526449
True      42975
dtype: int64

In [111]:
#단일 상품으로 고려
only_one = df[df["event_type"].isin(["cart", "purchase"])].drop_duplicates(["event_type", "product_id", "price", "user_id", "user_session"])
only_one

,Unnamed: 0,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session,depth1,depth2
113,162,2019-10-01 00:02:14 UTC,purchase,1004856,2053013555631882655,electronics.smartphone,samsung,130.76,543272936,8187d148-3c41-46d4-b0c0-9c08cd9dc564,electronics,smartphone
208,308,2019-10-01 00:04:37 UTC,purchase,1002532,2053013555631882655,electronics.smartphone,apple,642.69,551377651,3c80f0d6-e9ec-4181-8c5c-837a30be2d68,electronics,smartphone
285,442,2019-10-01 00:07:07 UTC,purchase,13800054,2053013557418656265,furniture.bathroom.toilet,santeri,54.42,555332717,1dea3ee2-2ded-42e8-8e7a-4e2ad6ae942f,furniture,toilet
361,574,2019-10-01 00:09:26 UTC,purchase,4804055,2053013554658804075,electronics.audio.headphone,apple,189.91,524601178,2af9b570-0942-4dcd-8f25-4d84fba82553,electronics,headphone
365,583,2019-10-01 00:09:33 UTC,cart,1002524,2053013555631882655,electronics.smartphone,apple,515.67,524325294,0b74a829-f9d7-4654-b5b0-35bc9822c238,electronics,smartphone
...,...,...,...,...,...,...,...,...,...,...,...,...
28932773,42448173,2019-10-31 23:55:21 UTC,purchase,47500017,2110937143172923797,construction.tools.light,puckator,20.59,514622109,5724116e-365b-4ac1-9d03-b8d66e1ccc7c,construction,light
28932835,42448271,2019-10-31 23:56:03 UTC,purchase,1003306,2053013555631882655,electronics.smartphone,apple,577.89,512717356,f35ac37c-9573-4e30-b3d9-c09bb0b95a2b,electronics,smartphone
28932847,42448292,2019-10-31 23:56:11 UTC,cart,1004240,2053013555631882655,electronics.smartphone,apple,1054.60,533892594,3a5a3b01-2ab1-4a1d-a202-30d336e0057b,electronics,smartphone
28932895,42448362,2019-10-31 23:56:53 UTC,purchase,1004240,2053013555631882655,electronics.smartphone,apple,1054.60,533892594,3a5a3b01-2ab1-4a1d-a202-30d336e0057b,electronics,smartphone


In [112]:
df_target = only_one.copy()
#구매 칼럼 생성
df_target["is_purchased"] = np.where(only_one["event_type"]=="purchase", 1, 0)

#한세션당 구입한 상품
df_target["is_purchased"] = df_target.groupby(["user_session", "product_id"])["is_purchased"].transform("max")

#카트에 넣고 구매했는지 중복값 제거
df_target = df_target.loc[df_target["event_type"]=="cart"].drop_duplicates(["user_session", "product_id", "is_purchased"])

In [113]:
df_target

,Unnamed: 0,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session,depth1,depth2,is_purchased
365,583,2019-10-01 00:09:33 UTC,cart,1002524,2053013555631882655,electronics.smartphone,apple,515.67,524325294,0b74a829-f9d7-4654-b5b0-35bc9822c238,electronics,smartphone,1
423,680,2019-10-01 00:11:00 UTC,cart,4804056,2053013554658804075,electronics.audio.headphone,apple,161.98,533624186,e5ac3caa-e6d5-4d6b-ae06-2c18cd9ca683,electronics,headphone,0
860,1325,2019-10-01 02:17:59 UTC,cart,1004833,2053013555631882655,electronics.smartphone,samsung,174.76,536415846,685b5b42-f597-4a69-ab4c-ef96a30bc454,electronics,smartphone,0
1089,1654,2019-10-01 02:19:36 UTC,cart,1005003,2053013555631882655,electronics.smartphone,huawei,258.21,513632293,f2cc68f7-39d1-4a50-9dcf-f2a0921bdfda,electronics,smartphone,1
1108,1684,2019-10-01 02:19:43 UTC,cart,1004750,2053013555631882655,electronics.smartphone,samsung,197.43,555110488,ce885079-4d92-4fe6-92a3-377c5a2d8291,electronics,smartphone,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
28932561,42447836,2019-10-31 23:52:56 UTC,cart,1004767,2053013555631882655,electronics.smartphone,samsung,242.63,542774966,957dc70c-31d3-42b7-aef0-2d2827c35251,electronics,smartphone,1
28932564,42447841,2019-10-31 23:52:57 UTC,cart,1004870,2053013555631882655,electronics.smartphone,samsung,275.25,550884173,32e8b23e-4bb6-4b67-b2eb-6f421dfd110a,electronics,smartphone,0
28932741,42448124,2019-10-31 23:55:07 UTC,cart,1005134,2053013555631882655,electronics.smartphone,apple,1889.07,566280291,20715c6f-d6fc-4ed4-9d48-c7c2b5d03602,electronics,smartphone,0
28932762,42448156,2019-10-31 23:55:14 UTC,cart,1003306,2053013555631882655,electronics.smartphone,apple,577.89,512717356,f35ac37c-9573-4e30-b3d9-c09bb0b95a2b,electronics,smartphone,1


In [114]:
#한 세션, 아이디당 카트에 담은 수
in_cart_ids = df.loc[df["event_type"] == "cart"].drop_duplicates("user_id")
in_cart = df.loc[df["user_id"].isin(in_cart_ids["user_id"])]
in_cart = in_cart.groupby(["user_session"])["event_type"].count().reset_index()
in_cart = in_cart.rename(columns={"event_type": "in_cart"})
in_cart

,user_session,in_cart
0,00000083-8816-4d58-a9b8-f52f54186edc,18
1,000001fd-1f89-45e8-a3ce-fe3218cabfad,11
2,000003eb-b63e-45d9-9f26-f229057c654a,1
3,00000809-9101-4e4b-9795-e6cbafccfe19,4
4,00000aaa-d774-49bc-9c31-0c9f6e1c2f0a,1
...,...,...
1799382,ffffa654-87e2-3a83-1a87-33c0480c3e36,7
1799383,ffffaa22-7177-4820-849e-3130c634ada3,4
1799384,ffffaa7e-6a84-47e8-aa8a-693be5f66120,1
1799385,ffffd89d-208c-28ec-da2f-6668c878fffc,1


In [115]:
df_target = df_target.merge(in_cart, on="user_session", how="left")
df_target["in_cart"] = df_target["in_cart"].fillna(0)
df_target[["user_id", "category_code", "brand", "user_session", "is_purchased", "in_cart"]].head()

,user_id,category_code,brand,user_session,is_purchased,in_cart
0,524325294,electronics.smartphone,apple,0b74a829-f9d7-4654-b5b0-35bc9822c238,1,4
1,533624186,electronics.audio.headphone,apple,e5ac3caa-e6d5-4d6b-ae06-2c18cd9ca683,0,4
2,536415846,electronics.smartphone,samsung,685b5b42-f597-4a69-ab4c-ef96a30bc454,0,7
3,513632293,electronics.smartphone,huawei,f2cc68f7-39d1-4a50-9dcf-f2a0921bdfda,1,16
4,555110488,electronics.smartphone,samsung,ce885079-4d92-4fe6-92a3-377c5a2d8291,1,10


In [116]:
df_target[df_target["user_id"] == 536415846][["category_code", "brand", "user_session", "is_purchased", "in_cart"]]

,category_code,brand,user_session,is_purchased,in_cart
2,electronics.smartphone,samsung,685b5b42-f597-4a69-ab4c-ef96a30bc454,0,7
2384,electronics.smartphone,samsung,ac58ce9c-89a9-4088-b0fb-4b1f19da718c,1,5


In [117]:
#구매자 비율
df_target[df_target["is_purchased"]== 1].shape[0] / df_target.shape[0]

0.4996265479317611

In [118]:
#비 구매자 비율
df_target[df_target["is_purchased"]== 0].shape[0] / df_target.shape[0]

0.5003734520682389

In [119]:
df_target.to_csv("./2019-Oct-depth-target.csv")